# Bengali Spelling Correction Using Noisy Chennel

TODO
```
[x] Generate Candidate Words
[x] Minimum Edit Distance
[x] Confusion Matrix
[x] Find Prior Prob
[x] Find Chennel Prob
[x] Smoothing
[x] Final Estimation (Real World)
```

In [ ]:
!pip install https://github.com/kpu/kenlm/archive/master.zip

     - 553.6 kB 11.5 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp310-cp310-linux_x86_64.whl size=3184345 sha256=f91cd2d79fe68cc467131b33d22c812021947f98f3e2b8b83b7957f04ede8403
  Stored in directory: /tmp/pip-ephem-wheel-cache-yirztd7s/wheels/a5/73/ee/670fbd0cee8f6f0b21d10987cb042291e662e26e1a07026462
Successfully built kenlm


In [ ]:
# PATHS
bn_characters   = "/content/drive/MyDrive/Spelling_Correction_Using_Noisy_Channel_Resources/bangla_datasets/bn_characters.txt"
bn_digits       = "/content/drive/MyDrive/Spelling_Correction_Using_Noisy_Channel_Resources/bangla_datasets/bn_digits.txt"
bn_sentences    = "/content/drive/MyDrive/Spelling_Correction_Using_Noisy_Channel_Resources/bangla_datasets/bn_sentences.txt"
bn_dictionary   = "/content/drive/MyDrive/Spelling_Correction_Using_Noisy_Channel_Resources/bangla_datasets/bn_dictionary.txt"
bn_arpa         = "/content/drive/MyDrive/Spelling_Correction_Using_Noisy_Channel_Resources/bangla_datasets/text.arpa"
bn_del_path     = "/content/drive/MyDrive/Spelling_Correction_Using_Noisy_Channel_Resources/bangla_datasets/bn_del.txt"
bn_ins_path     = "/content/drive/MyDrive/Spelling_Correction_Using_Noisy_Channel_Resources/bangla_datasets/bn_ins.txt"
bn_sub_path     = "/content/drive/MyDrive/Spelling_Correction_Using_Noisy_Channel_Resources/bangla_datasets/bn_sub.txt"

In [ ]:
# IMPORTS
import time
import numpy as np
import nltk
import kenlm
from nltk.util import ngrams
from collections import Counter

In [ ]:
# KENLM MODEL OBJECT
kenlm_model = kenlm.Model(bn_arpa)

In [ ]:
# REQUIRED LIBRARY (one time)
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1. Generate Candidate Words

In [ ]:
# GET ALPHABETS
with open(bn_characters, "r") as file:
    characters = file.read().split("\t")
print(characters)

['ঀ', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'ঌ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'ঽ', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ৄ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ৗ', 'ড়', 'ঢ়', 'য়', 'ৠ', 'ৡ', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯', 'ৰ', 'ৱ', '৲', '৳', '৴', '৵', '৶', '৷', '৸', '৹', '৺', '৻', 'ৼ', '৽', '৾', '']


In [ ]:
# VALUE INDEX PAIR OF CHARACTERS
char_lookup = dict(zip(characters, list(range(len(characters)))))

# TEST
char_lookup["ই"]

6

In [ ]:
# GET DIGITS
with open(bn_digits, "r") as file:
    digits = file.read().split("\t")
print(digits)

['০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯']


In [ ]:
# ALPAHBETS
bn_alphabets = characters + digits
print(bn_alphabets)

['ঀ', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'ঌ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'ঽ', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ৄ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ৗ', 'ড়', 'ঢ়', 'য়', 'ৠ', 'ৡ', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯', 'ৰ', 'ৱ', '৲', '৳', '৴', '৵', '৶', '৷', '৸', '৹', '৺', '৻', 'ৼ', '৽', '৾', '', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯']


In [ ]:
# READ DICTIONARY
with open(bn_dictionary, "r") as file:
    bn_dictionary = file.read().split("\t")

# TEST
bn_dictionary[:10]

['পাবনার',
 'সুজানগর',
 'পৌরসভার',
 'মসজিদপাড়া',
 'জামে',
 'মসজিদের',
 'নতুন',
 'ভবন',
 'নির্মাণকে',
 'কেন্দ্র']

In [ ]:
# FIND ALL CANDIDATES
def find_candidate_words(word):                     # get candidate words with single transformation
    ''' Find all candidate words for given misspelled word by using {insetion, deletion, transpose, substitution}
    Args:   word (string) = misspelled word
    Return: (list) a list containing all candidate words '''
    candidate_words = []
    for char in bn_alphabets:
        for idx in range(len(word)+1):              # Insert candidate
            candidate_words.append(word[:idx]+char+word[idx:])
        for idx in range(len(word)):                # Substitution candidate
            candidate_words.append(word[:idx]+char+word[idx+1:])
    for idx in range(len(word)):                    # Deletion candidate
        candidate_words.append(word[:idx]+word[idx+1:])
    if(len(word)>1):                                # Transpose candidate
        for idx in range(len(word)-1):
            candidate_words.append(word[:idx]+word[idx+1]+word[idx]+word[idx+2:])
    return candidate_words

In [ ]:
# FIND VALID CANDIDATES
def find_valid_candidates(word, med):    # single edit distance candidates
    ''' Fina all candidates within given edit distance
        Args:   word: (string) error word
                med: (int) minimum edit distance
        Return: (list) all valid candidate words '''

    candidates = []
    candidate_words = find_candidate_words(word)    # for edit distance 1 (initial)
    for i in range(med-1):                          # for edit distance > 1
        for candidate_word in candidate_words:
            candidate_words = set(find_candidate_words(candidate_word)).union(candidate_words)
    valid_candidates = list(set(candidate_words).intersection(set(bn_dictionary))) # match all generated candidate words with dictionary
    return valid_candidates

In [ ]:
# TEST
starttime = time.time()
_temp = "কেন্দ্র"
candidates = find_valid_candidates(_temp, 1)
print(f"Original word: {_temp}")
print(candidates)
print(f'{(time.time()-starttime)*10**3} ms')

Original word: কেন্দ্র
['সেন্দ্র', 'কেন্দ্রী', 'কেন্দ্রই', 'কেন্দের', 'কেন্দ্রি', 'কেন্দ্র', 'কেন্দ্রে', 'কেন্দ্রও', 'কেন্দ্রর']
1733.8333129882812 ms


## 2. Minimum Edit Distance

In [ ]:
def get_minimum_edit_distance(s1, s2):
    ''' Calculate the minimum edit distance between two strings
        Args:   s1 (string) = first string; error word
                s2 (string) = second string; candidate word
        Return: (int) minimum edit distance to transform from s1 to s2,
                (2D numpy array) calculated cost matrix '''

    m = len(s1)                         # length of source string
    n = len(s2)                         # length of target string
    mat = np.zeros((m+1, n+1))          # 2D array of dimention (n+1 X m+1) filled with 0's

    for i in range(1, m+1):             # Initializing row 1
        mat[i, 0] = mat[i-1, 0] + 1
    for j in range(1, n+1):             # Initializing column 1
        mat[0, j] = mat[0, j-1] + 1

    # Iteration
    for i in range(1, m+1):
        for j in range(1, n+1):
            sc = 2 if s1[i-1] != s2[j-1] else 0
            cost = min([mat[i-1, j] + 1,        # deletion
                        mat[i, j-1] + 1,        # insertion
                        mat[i-1, j-1] + sc])    # substitution
            mat[i, j] = cost                    # update cost

    return int(mat[-1,-1]), mat

In [ ]:
def get_edit_sequence(s, d):
    ''' Calculate edit sequence from the given correct word and typo
    Args:   s: (string) typo or source word
            d: (string) candidate or destination word
    Return: (list) sequence of transformation in reversed order <transformation> <source> <destination> '''

    _, m = get_minimum_edit_distance(s, d)
    s = '#' + s
    d = '#' + d
    sequence = list()
    i, j = len(s)-1, len(d)-1
    while True:
        if i>0 and j>0:     # diagonal element present
            if m[i,j] == m[i-1,j-1] and s[i] == d[j]:
                sequence.append('')
                i -= 1
                j -= 1
            elif m[i,j] == m[i-1,j-1]+2 and s[i] != d[j]:
                sequence.append('s'+d[j]+s[i])
                i -= 1
                j -= 1
            elif m[i,j] == m[i-1,j]+1:
                sequence.append('d'+d[j]+s[i])
                i -= 1
            else:
                sequence.append('i'+d[j]+s[i])
                j -= 1
        elif i==0 and j==0:
            break
        elif i == 0:
            sequence.append('i'+d[j]+s[i])
            j -= 1
        elif j == 0:
            sequence.append('d'+d[j]+s[i])
            i -= 1
    return sequence # [<type>,<destination>,<source>]

In [ ]:
# TEST
s = "কলকাতা"
d = "কলিকাতা"
get_edit_sequence(s, d)

['', '', '', '', 'iিল', '', '']

## 3. Confusion Matrix

In [ ]:
# INS MATRIX
bn_ins = list()
with open(bn_ins_path, "r") as file:
    temp = file.read().split("\n")
    for t in temp:
        bn_ins.append(t.split(" "))

In [ ]:
# INS MATRIX
bn_del = list()
with open(bn_del_path, "r") as file:
    temp = file.read().split("\n")
    for t in temp:
        bn_del.append(t.split(" "))

In [ ]:
# INS MATRIX
bn_sub = list()
with open(bn_sub_path, "r") as file:
    temp = file.read().split("\n")
    for t in temp:
        bn_sub.append(t.split(" "))

In [ ]:
# TEST
for a in bn_ins:
    for b in a:
        print(b, end=" ")
    print()

## 4. Prior Probability

In [ ]:
# REMOVE PUNCHUATIONS
import string
strip_chars = string.punctuation
def remove_punc(sentence):
    return sentence.translate(str.maketrans('', '', string.punctuation)).lower()

In [ ]:
# RETURNS UNIGRAM SCORE
def get_score(s):
    return [10**prob for prob, _, _ in kenlm_model.full_scores(remove_punc(s), bos=False, eos=False)]

In [ ]:
get_score("আমার নাম, শুভাশীষ")

[0.0005424991727568429, 0.005281700737824817, 5.67493131376058e-07]

In [ ]:
def prior_prob(word):   # SMOOTHED PRIOR PROBABILITY
    return 10**kenlm_model.score(remove_punc(word), bos=False, eos=False)

## 5. Channel Probability

In [ ]:
# READ ALL SENTENCES
with open(bn_sentences, "r") as file:
    sentences = file.read().split("\n")

# TEST
sentences[:5]

['',
 'বিশ্বজিত্ বলেন, ‘এই সময় হ্যান্ড মাইকে তিনি পুলিশের কথা শুনতে পান',
 'এর আগে মঙ্গলবার লে.কর্নেল আবুল কালাম আজাদের পরীক্ষা-নীরিক্ষার রিপোর্ট চিকিৎসকদের বোর্ড পর্যালোচনা করে',
 'বিমানের চিফ অব টেকনিক্যাল ক্যাপ্টেন ফজল মাহমুদ চৌধুরীর নেতৃত্বে ৪ সদস্যের কমিটির অপর সদস্যরা হলেন- চিফ অব ফ্লাইট সেফটি ক্যাপ্টেন সোয়েব চৌধুরী, ডেপুটি চিফ অব ইঞ্জিনিয়ার মোহাম্মদ হানিফ ও ম্যানেজার (কোয়ালিটি অ্যাসুরেন্স) নিরঞ্জন রায়',
 'জেলার ভোগড়ায় বিআরটি প্রকল্পের কাজ পরিদর্শন শেষে সাংবাদিকদের বিভিন্ন প্রশ্নের জবাবে তিনি এ কথা বলেন']

In [ ]:
# COUNT MATRIX
def get_count(lexem):
    unigram_count = dict()
    bigram_count = dict()
    for sentence in sentences:
        bigrams = ["".join(a) for a in list(ngrams(lexem, n=2))]
        unigrams = ["".join(a) for a in list(ngrams(lexem, n=1))]
        bcount = Counter(bigrams)
        ucount = Counter(unigrams)

        for key in bcount:  # Count all bigrams
            try:
                bigram_count[key] += bcount[key]
            except:
                bigram_count[key] = bcount[key]

        for key in ucount:  # count all unigrams
            try:
                unigram_count[key] += ucount[key]
            except:
                unigram_count[key] = ucount[key]

    return unigram_count, bigram_count

In [ ]:
get_count("কলিকাতা")

({'ক': 1480742, 'ল': 740371, 'ি': 740371, 'া': 1480742, 'ত': 740371},
 {'কল': 740371,
  'লি': 740371,
  'িক': 740371,
  'কা': 740371,
  'াত': 740371,
  'তা': 740371})

In [ ]:
# CHAR PROBAIBLITY
def char_probability(x, y, type):
    if x == "#":
        xi = -1
    else:
        xi = char_lookup[x]

    if y == "#":
        yi = -1
    else:
        yi = char_lookup[y]

    # print(f"{type}({xi},{yi})")
    uni_count, bi_count = get_count(x+y)
    if type.lower() == "d":
        return int(bn_del[xi][yi])/int(bi_count[x+y])
    elif type.lower() == "i":
        return int(bn_ins[yi][xi])/int(uni_count[y])
    elif type.lower() == "s":
        return int(bn_sub[xi][yi])/int(uni_count[y])
    return -1

In [ ]:
char_probability("ক","ত","s")

1.35067418902145e-05

In [ ]:
# CHANNEL PROBABILITY
def get_channel_probability(word, candidate):
    edit_sequence = get_edit_sequence(word, candidate)
    probability = 0
    for sequence in edit_sequence:
        if sequence != '':
            sequence_list = list(sequence)
            probability += char_probability(sequence_list[2], sequence_list[1], sequence_list[0])
    return probability

In [ ]:
# TEST
s = "কলকাতা"
d = "কলিকাতা"
get_channel_probability(s,d)

4.0520225670643505e-06

## 6. Final Estimation (real word)

In [ ]:
# MAKE N-GRAM
def extract_ngrams(sentence, n):
    n_grams = ngrams(nltk.word_tokenize(sentence), n)
    # return [ ' '.join(grams) for grams in n_grams]
    return [grams for grams in n_grams]

In [ ]:
# TEST
sent = "কলিকাতা সহ দক্ষিণবঙ্গের সব জেলাতেই বৃষ্টির সঙ্গে থাকবে দমকা ঝোড়ো হাওয়া"
extract_ngrams(sent,2)

[('কলিকাতা', 'সহ'),
 ('সহ', 'দক্ষিণবঙ্গের'),
 ('দক্ষিণবঙ্গের', 'সব'),
 ('সব', 'জেলাতেই'),
 ('জেলাতেই', 'বৃষ্টির'),
 ('বৃষ্টির', 'সঙ্গে'),
 ('সঙ্গে', 'থাকবে'),
 ('থাকবে', 'দমকা'),
 ('দমকা', 'ঝোড়ো'),
 ('ঝোড়ো', 'হাওয়া')]

In [ ]:
get_score(sent)

[5.810162642229453e-07,
 0.00018174584975182511,
 9.68103654932087e-07,
 0.022797164381223846,
 0.00018543514697413207,
 5.196947266935102e-05,
 0.01996643819430989,
 0.0004856324165167078,
 3.0672701856974783e-06,
 8.651270801033902e-08,
 0.22094757697193174]

In [ ]:
state = kenlm.State()
state2 = kenlm.State()

def get_lm_prob(sent):
    accum = 0.0
    for word in sent.split(" "):
        # print(word)
        accum += kenlm_model.BaseScore(state, word, state2)
    return 10**accum

In [ ]:
get_lm_prob("সঙ্গে থাকবে")

সঙ্গে
থাকবে


5.145360019930381e-07

In [ ]:
# GENERATE ALL CANDIDATES
def candidate_generator(sent, med):
    '''
    Args:   sent (str) = given sentence to check
            med (int) = minimum edit distance to check for candidates
    Return: (list of list) = list of candidate words according to their position '''
    candidates = []
    for word in remove_punc(sent).strip().split(" "):   # iterate through each words
        candidates.append(find_valid_candidates(word, med))
    return candidates

In [ ]:
# TEST
candidate_generator("দমকা ঝোড়ো হাওয়া", 1)

[['দমকাল', 'দম্কা', 'দমকল', 'দমকায়', 'দেকা', 'দমকম', 'দমকা', 'অমকা'],
 ['ঝড়ো', 'ঝোড়ো'],
 ['হাওয়া',
  'চাওয়া',
  'দাওয়া',
  'ধাওয়া',
  'পাওয়া',
  'মাওয়া',
  'খাওয়া',
  'হওয়া',
  'যাওয়া',
  'হাওয়ার']]

In [ ]:
alpha = 0.95 # config

In [ ]:
# GENERATE ALL SENTENCE PROBABILITY
def sentence_probability(sent, med):
    candidates_list = candidate_generator(sent, med)
    sent_prob = {}
    for i in range(len(candidates_list)):
        for candidate in candidates_list[i]:
            edit_prob = alpha   # when P(w|w)
            prior_prob = 1
            candidate_sentence = " ".join(sent.split()[:i]+[candidate]+sent.split()[i+1:])
            print("= "*30)      # test
            print(candidate_sentence)   # test
            if sent.split()[i] != candidate:
                edit_prob = get_channel_probability(sent.split()[i], candidate)
                print(f"P({sent.split()[i]}|{candidate}): {edit_prob}")      # test
            for ngram in extract_ngrams(candidate_sentence, 2):
                prior_prob *= get_lm_prob(" ".join(ngram))
                print(f"{ngram}: {get_lm_prob(' '.join(ngram))}")     # test
            sent_prob[candidate_sentence] = (edit_prob * prior_prob)
        # break     # test
    return sent_prob

In [ ]:
# CANDIDATE WORDS STATUS
sent = "মাঝরি বৃষ্টির পূর্বাভাস রয়েছে"
prob = sentence_probability(sent, 1)

= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
মাঝির বৃষ্টির পূর্বাভাস রয়েছে
P(মাঝরি|মাঝির): 1.08053935121716e-05
('মাঝির', 'বৃষ্টির'): 1.3837358720736688e-09
('বৃষ্টির', 'পূর্বাভাস'): 2.417338660152965e-09
('পূর্বাভাস', 'রয়েছে'): 8.277517077940962e-10
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
মাঝরি বৃষ্টির পূর্বাভাস রয়েছে
('মাঝরি', 'বৃষ্টির'): 2.99104485238234e-11
('বৃষ্টির', 'পূর্বাভাস'): 2.417338660152965e-09
('পূর্বাভাস', 'রয়েছে'): 8.277517077940962e-10
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
মাঝারি বৃষ্টির পূর্বাভাস রয়েছে
P(মাঝরি|মাঝারি): 1.35067418902145e-06
('মাঝারি', 'বৃষ্টির'): 3.282677353071633e-09
('বৃষ্টির', 'পূর্বাভাস'): 2.417338660152965e-09
('পূর্বাভাস', 'রয়েছে'): 8.277517077940962e-10
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
মারি বৃষ্টির পূর্বাভাস রয়েছে
P(মাঝরি|মারি): 0.0
('মারি', 'বৃষ্টির'): 2.2994867772706185e-10
('বৃষ্টির', 'পূর্বাভাস'): 2.417338660152965e-09
('পূর্বাভাস', 'রয়েছে'): 8.2

In [ ]:
sorted(prob.items(), key=lambda x: x[1], reverse=True)

[('মাঝরি বৃষ্টির পূর্বাভাস রয়েছে', 5.685702266859855e-29),
 ('মাঝির বৃষ্টির পূর্বাভাস রয়েছে', 2.9917918214307036e-32),
 ('মাঝারি বৃষ্টির পূর্বাভাস রয়েছে', 8.871894788162864e-33),
 ('মাঝরি বৃষ্টি পূর্বাভাস রয়েছে', 3.404490235863123e-33),
 ('মাঝরি সৃষ্টির পূর্বাভাস রয়েছে', 1.0286327747965633e-33),
 ('মাঝর বৃষ্টির পূর্বাভাস রয়েছে', 6.466973724891247e-34),
 ('মাঝরি বৃষ্টির পূর্বাভাস রয়েছেন', 2.307464760258603e-34),
 ('মাঝরি বৃষ্টির পূর্বাভাসে রয়েছে', 1.9039066289874622e-34),
 ('মাঝরি বৃষ্টির পূর্বাভাস হয়েছে', 1.2235411136286852e-34),
 ('মাঝরি বৃষ্টির পুর্বাভাস রয়েছে', 6.694439526030102e-36),
 ('মাঝরি দৃষ্টির পূর্বাভাস রয়েছে', 1.3005597260045802e-36),
 ('মাঝরি বৃষ্টির পূর্বাবাস রয়েছে', 1.1633484585835033e-36),
 ('মাঝরি বিষ্টির পূর্বাভাস রয়েছে', 3.4210955357459406e-37),
 ('মাঝরি বৃষ্টির পূর্বাভাসও রয়েছে', 2.033221421032108e-37),
 ('মাঝরি বৃষ্টির পূর্বভাস রয়েছে', 1.2376047431739396e-37),
 ('মাঝরি বৃষ্টিত পূর্বাভাস রয়েছে', 6.769012557358279e-38),
 ('মাঝরি বৃষ্টিও পূর্বাভাস রয়ে

## 7. Testing

In [ ]:
print("hello world")

hello world


In [ ]:
get_lm_prob("আমি শুভাশীষ")

8.397126160390632e-10

In [ ]:
get_lm_prob("পূর্বাভাস রয়েছে")

8.277517077940962e-10